# DefDAP example notebook
This notebook will outline basic usage of DefDAP

## Load in packages

DefDAP is split into modules for processing EBSD and HRDIC. There are also mudules for manpulating orientations (defdap.quat) and creating custom figures (defdap.plotting) which is introduced later. We also import some of the usual suspects of the python scienctific stack, numpy and matplotlib.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
from defdap.quat import Quat

# try tk, qt, osx (if using mac) or notebook for interactive plots. If none work, use inline
%matplotlib tk

## Load in a HRDIC map

In [3]:
dicFilePath = "tests/data/"
dicMap = hrdic.Map(dicFilePath, "testDataDIC.txt")

Loaded DaVis 8.4.0 data (dimensions: 300 x 200 pixels, sub-window size: 12 x 12 pixels)


### Set the scale of the map
This is read as the pixel size in the DIC pattern images.

In [4]:
fieldWidth = 20 # microns
numPixels = 2048
pixelSize = fieldWidth / numPixels

dicMap.setScale(pixelSize)

### Plot the map with a scale bar

In [5]:
dicMap.plotMaxShear(vmin=0, vmax=0.10, plotScaleBar=True)

### Crop the map
HRDIC maps often contain spurious data at the edges which should be removed before performing any analysis. The crop is defined by the number of points to remove from each edge of the map, where xMin, xMax, yMin and yMax are the left, right, top and bottom edges respectively. Note that the test data doesn not require cropping as it is a subset of a larger dataset.

In [6]:
dicMap.setCrop(xMin=0, xMax=0, yMin=0, yMax=0)

### Set the location of the DIC pattern images  
The pattern images are used later to define the position of homologous material points. The path is relative to the directory set when loading in the map. The second parameter is the pixel binning factor of the image relative to the DIC sub-region size i.e. the number of pixels in the image across a single datapoint in the DIC map. We recommend binning the pattern images by the same factor as the DIC sub-region size, doing so enhances the contrast between microstructure features.

In [7]:
# set the path of the pattern image, this is relative to the location of the DIC data file
dicMap.setPatternPath("testDataPat.bmp", 1)

## Load in an EBSD map
Set the crystal symmetry and (optially) load in a set of slip systems defined in files stroed in the directory defdap/slip_systems by default. The orientation in the EBSD are converted to a quaternion representation so calculation can be applied later

In [8]:
ebsdFilePath = "tests/data/testDataEBSD"
crystalSymmetry = "cubic"

ebsdMap = ebsd.Map(ebsdFilePath, crystalSymmetry)
ebsdMap.loadSlipSystems("cubic_fcc")
ebsdMap.buildQuatArray()

Loaded EBSD data (dimensions: 359 x 243 pixels, step size: 0.12 um)
Finished building quaternion array           


### Plot the EBSD map
Using an Euler colour mapping or inverse pole figure colouring with the sample reference direction passed as a vector.

In [9]:
ebsdMap.plotEulerMap(plotScaleBar=True)

In [10]:
ebsdMap.plotIPFMap([1,0,0], plotScaleBar=True)

### Detect grains in the EBSD
This is done in two stages: first bounaries are detected in the map as any point with a misorientation to a neighbouring point greater than a critical value (boundDef in degrees). A flood fill type algorithm is then applied to segment the map into grains, with any grains containining fewer than a critical number of pixels removed (minGrainSize in pixels). The data e.g. orientations associated with each grain are then stored (referenced strictly, the data isn't stored twice) in a grain object and a list of the grains is stored in the EBSD map (named grainList). This allows analysis routines to be applied to each grain in a map in turn.

In [11]:
ebsdMap.findBoundaries(boundDef=8)
ebsdMap.findGrains(minGrainSize=10)

Finished finding grain boundaries           
Finished finding grains           


### Single grain analysis
The 'locateGrainID' method allows interactive selection of a grain of intereset to apply any analysis to. Clicking on grains in the map will highlight the grain and print out the grain ID (position in the grain list) of the grain.

In [15]:
ebsdMap.locateGrainID()

Grain ID: 26


A built-in example is to calculate the average orientation of the grain and plot this orientatioan in a IPF

In [13]:
grainID = 48
grain = ebsdMap[grainID]
grain.calcAverageOri()  # stored as a quaternion named graian.refOri
print(grain.refOri)
grain.plotRefOri(direction=[0, 0, 1])

[0.8385, -0.0156, 0.2253, 0.4960]


A second built-in example is to calcuate the grain misorientation, specifically the grain reference orientation deviation (GROD). This shows another feature of the 'locateGrainID' method, which stores the ID of the last selected grain in a variable called currGrainId in the EBSD map.

In [16]:
grain = ebsdMap[ebsdMap.currGrainId]
grain.buildMisOriList()
grain.plotMisOri(plotScaleBar=True, vmin=0, vmax=5)

### Multi grain analysis
Once an analysis routine has been prototyped for a single grain it can be applied to all the grains in a map using a loop over the grains and any results added to a list for use later. Of couse you could also apply to a smaller subset of grains as well.

In [20]:
grainAvOris = []
for grain in ebsdMap:
    grain.calcAverageOri()
    grainAvOris.append(grain.refOri)

# Plot all the grain orientations in the map
Quat.plotIPF(grainAvOris, [0, 0, 1], ebsdMap.crystalSym, marker='o', s=10)
plt.tight_layout()

Some common grain analysis routines are built into the EBSD map object, including:

In [21]:
ebsdMap.calcGrainAvOris()

Finished calculating grain mean orientations           


In [22]:
ebsdMap.calcGrainMisOri()
ebsdMap.plotMisOriMap(vmin=0, vmax=5, plotGBs=True, plotScaleBar=True)

Finished calculating grain misorientations           


There are also methods for plotting KAM, GND density, phases and boundaries. All of the plotting functions in DefDAP use the same parameters to modify the plot, examples seen so far are 'plotGBs', 'plotScaleBar', 'vmin', 'vmax'.

## Linking the HRDIC and EBSD
### Define homologous points
To register the two datasets, homologous points (points at the same material location) within each map are used to estimate a transformation between the two frames the data are defined in. The homologous points are selected manually using an interactive tool within DefDAP. To select homologous call the method 'setHomogPoint' on each of the data maps, which will open a plot window with a button labelled 'save point' in the bottom right. You select a point by right clicking on the map, adjust the position with the arrow  and accept the point by with the save point button. Then select the same location in the other map. Note that as we set the location of the pattern image for the HRDIC map that the points can be selected on the pattern image rather than the strain data.

In [30]:
dicMap.setHomogPoint(display="pattern")

In [31]:
ebsdMap.setHomogPoint()

The points are stored as a list of tuples (x, y) in each of the maps. This means the points can be set from previous values 

In [28]:
dicMap.homogPoints

[(36, 72), (279, 27), (162, 174), (60, 157)]

In [29]:
ebsdMap.homogPoints

[(68, 95), (308, 45), (191, 187), (89, 174)]

Here are some example homologous points for this data, after setting these by running the cells below you can view the locations in the maps by running the 'setHomogPoint' methods (above) again

In [26]:
dicMap.homogPoints = [
    (36, 72), 
    (279, 27), 
    (162, 174), 
    (60, 157)
]

In [27]:
ebsdMap.homogPoints = [
    (68, 95), 
    (308, 45), 
    (191, 187), 
    (89, 174)
]

### Link the maps
Finally the two data maps are linked. The type of transform between the two frames can be affine, projective, polynomial.

In [32]:
dicMap.linkEbsdMap(ebsdMap, transformType="affine")
# dicMap.linkEbsdMap(ebsdMap, transformType="polynomial", order=2)

### Show the transformation

In [33]:
from skimage import transform as tf

data = np.zeros((2000, 2000), dtype=float)
data[500:1500, 500:1500] = 1.
dataWarped = tf.warp(data, dicMap.ebsdTransform)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(data)
ax2.imshow(dataWarped)

### Segment into grains
The HRDIC map can now be segmented into grains using the grain boundaries detected in the EBSD map. Analysis rountines can then be applied to individual grain, as with the EBSD grains. The grain finding process will also attempt to link the grains between the EBSD and HRDIC and each grain in the HRDIC has a reference ('ebsdGrain') to the corrosponding grain in the EBSD map.

In [34]:
dicMap.findGrains(minGrainSize=10)

Finished finding grains           


In [35]:
dicMap.plotMaxShear(vmin=0, vmax=0.1, plotScaleBar=True, plotGBs=True)

### Built-in plots

In [36]:
plot = dicMap.plotMaxShear(
    vmin=0, vmax=0.1, plotScaleBar=True,
    plotGBs=True, dilateBoundaries=True
)

In [37]:
plot = ebsdMap.plotEulerMap(
    plotScaleBar=True, plotGBs=True,
    highlightGrains=[10, 20, 45], highlightAlpha=0.9, highlightColours=['y']
)

In [38]:
dicMap.locateGrainID()

Grain ID: 41


In [40]:
dicGrainID = 41
dicGrain = dicMap[dicGrainID]

plot = dicGrain.plotMaxShear(
    plotScaleBar=True, plotSlipTraces=True, plotSlipBands=True
)

Number of bands detected: 2


### Create your own

In [41]:
from defdap.plotting import MapPlot, GrainPlot, HistPlot

In [43]:
mapData = dicMap.e11
mapData = dicMap.crop(mapData)

plot = MapPlot.create(
    dicMap, mapData,
    vmin=-0.1, vmax=0.1, plotColourBar=True, cmap="seismic",
    plotGBs=True, dilateBoundaries=True, boundaryColour='black'
)


In [44]:
plot.addScaleBar()

### Functions for grain averaging and grain segmentation

In [45]:
plot = dicMap.plotGrainDataMap(
    mapData,
    vmin=-0.06, vmax=0.06, plotColourBar=True,
    cmap="seismic", cLabel="Axial strain ($e_11$)",
    plotScaleBar=True
)


In [46]:
plot.addGrainBoundaries(dilate=True, colour="white")

In [47]:
plot = dicMap.plotGrainDataIPF(
    np.array((1,0,0)), mapData, marker='o',
    vmin=-0.06, vmax=0.06, plotColourBar=True, 
    cLabel="Axial strain ($e_11$)", cmap="seismic",
)


In [48]:
dicGrainID = 41
dicGrain = dicMap[dicGrainID]

plot = dicGrain.plotGrainData(
    mapData, 
    vmin=-0.1, vmax=0.1, plotColourBar=True, 
    cLabel="Axial strain ($e_11$)", cmap="seismic",
    plotScaleBar=True
)


In [49]:
plot.addSlipTraces()

In [52]:
dicMap.locateGrainID(vmax=0.1)

### Composite plots

In [50]:
from matplotlib import gridspec

In [52]:
fig = plt.figure(figsize=(8, 4)) 
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1],
                       wspace=0.15, hspace=0.15, 
                       left=0.02, right=0.98,
                       bottom=0.12, top=0.95) 
ax0 = plt.subplot(gs[:, 0])
ax1 = plt.subplot(gs[0, 1])
ax2 = plt.subplot(gs[1, 1])


# add a strain map
plot0 = dicMap.plotMaxShear(
    ax=ax0, fig=fig, 
    vmin=0, vmax=0.08, plotScaleBar=True, 
    plotGBs=True, dilateBoundaries=True
)

# add an IPF of grain orientations
dicOris = []
for grain in dicMap:
    if len(grain) > 20:
        dicOris.append(grain.refOri)
plot1 = Quat.plotIPF(
    dicOris, np.array((1,0,0)), 'cubic', 
    ax=ax1, fig=fig, s=10
)

# add histrogram of strain values
plot2 = HistPlot.create(
    dicMap.crop(dicMap.eMaxShear),
    ax=ax2, fig=fig,
    plotType="log", bins=50, range=(0,0.06)
)
plot2.ax.set_xlabel("Effective shear strain")

Text(0.5, 18.72222222222218, 'Effective shear strain')

In [121]:
plt.savefig("/Users/mbcx9ma4/Desktop/defdap_plotting_update/test_map_plot.png", dpi=200)

In [54]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(6, 4))

dicGrainID = 41
dicGrain = dicMap[dicGrainID]

# add a strain map
plot0 = dicGrain.plotMaxShear(
    ax=ax0, fig=fig, 
    vmin=0, vmax=0.08, plotScaleBar=True,
    plotSlipTraces=True
)


# add a misorientation
ebsdGrain = dicGrain.ebsdGrain
plot1 = ebsdGrain.plotMisOri(component=0, ax=ax1, fig=fig, vmin=0, vmax=1, cLabel="GROD", plotScaleBar=True)


# add an IPF
plot2 = ebsdGrain.plotOriSpread(
    direction=np.array((1,0,0)), c='b', s=1, alpha=0.2,
    ax=ax2, fig=fig
)
ebsdGrain.plotRefOri(
    direction=np.array((1,0,0)), c='k', s=100, plot=plot2
)


# add histrogram of strain values
# grainData
# plot2 = HistPlot.create(
#     dicMap.crop(dicMap.max_shear),
#     ax=ax2, fig=fig,
#     plotType="log", bins=50, range=(0,0.06)
# )
# plot2.ax.set_xlabel("Effective shear strain")


plt.tight_layout()